## Importing Libraries

In [2]:
import tensorflow as tf
import keras
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
from keras.models import load_model

## Retraining MobileNet for generating custom model via Transfer Learning

## Non-optimised

### Data Pre-processing

In [3]:
# Data fetch
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                 validation_split=0.2)

train_generator=train_datagen.flow_from_directory("./scrapped images",
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=10,
                                                 class_mode='binary',
                                                 shuffle=True,
                                                 subset='training')

validation_generator=train_datagen.flow_from_directory("./scrapped images",
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=10,
                                                 class_mode='binary',
                                                 shuffle=True,
                                                 subset='validation')

Found 736 images belonging to 2 classes.
Found 184 images belonging to 2 classes.


### Defining Model Architecture

In [ ]:
# Loading the MobileNet model
base_model = MobileNetV2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
) 

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(224, 224, 3))


# The base model contains batchnorm layers. We want to keep them in inference 
# mode when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1,activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


### Model Training

In [ ]:
#last layers training
model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()]
)

epochs = 25

model.fit(x=train_generator,
          steps_per_epoch = train_generator.n//train_generator.batch_size,
          validation_data = validation_generator, 
          validation_steps = validation_generator.n//validation_generator.batch_size,
          epochs = epochs)


Epoch 1/25
23/23 [==============================] - 305s 12s/step - loss: 0.8362 - binary_accuracy: 0.5068 - val_loss: 0.7355 - val_binary_accuracy: 0.5375
Epoch 2/25
23/23 [==============================] - 5s 215ms/step - loss: 0.7371 - binary_accuracy: 0.5145 - val_loss: 0.6949 - val_binary_accuracy: 0.5500
Epoch 3/25
23/23 [==============================] - 5s 217ms/step - loss: 0.7138 - binary_accuracy: 0.5545 - val_loss: 0.6684 - val_binary_accuracy: 0.5813
Epoch 4/25
23/23 [==============================] - 5s 218ms/step - loss: 0.6747 - binary_accuracy: 0.5955 - val_loss: 0.6469 - val_binary_accuracy: 0.6500
Epoch 5/25
23/23 [==============================] - 5s 218ms/step - loss: 0.6409 - binary_accuracy: 0.6046 - val_loss: 0.6228 - val_binary_accuracy: 0.6812
Epoch 6/25
23/23 [==============================] - 5s 215ms/step - loss: 0.6045 - binary_accuracy: 0.6951 - val_loss: 0.6132 - val_binary_accuracy: 0.7188
Epoch 7/25
23/23 [==============================] - 5s 217ms/ste

### Model Fine Tuning

In [ ]:
# fine tuning
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-6),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()]
)

epochs = 10
model.fit_generator(generator=train_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    validation_data = validation_generator, 
                    validation_steps = validation_generator.n//validation_generator.batch_size,
                    epochs = epochs)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


23/23 [==============================] - 11s 315ms/step - loss: 0.3520 - binary_accuracy: 0.8651 - val_loss: 0.3910 - val_binary_accuracy: 0.8500
Epoch 2/10
23/23 [==============================] - 6s 278ms/step - loss: 0.3254 - binary_accuracy: 0.8974 - val_loss: 0.3704 - val_binary_accuracy: 0.8438
Epoch 3/10
23/23 [==============================] - 7s 277ms/step - loss: 0.3254 - binary_accuracy: 0.8866 - val_loss: 0.3714 - val_binary_accuracy: 0.8562
Epoch 4/10
23/23 [==============================] - 7s 280ms/step - loss: 0.3224 - binary_accuracy: 0.8714 - val_loss: 0.3614 - val_binary_accuracy: 0.8750
Epoch 5/10
23/23 [==============================] - 6s 276ms/step - loss: 0.3130 - binary_accuracy: 0.8806 - val_loss: 0.3650 - val_binary_accuracy: 0.8438
Epoch 6/10
23/23 [==============================] - 6s 269ms/step - loss: 0.2779 - binary_accuracy: 0.9029 - val_loss: 0.3430 - val_binary_accuracy: 0.8500
Epoch 7/10
23/23 [==============================] - 6s 275ms/step - loss: 

### Saving the model 

In [ ]:
model.save("Final_modelN_89_87.h5")

## Optimised

### Data Pre-processing

In [4]:
# Data fetch
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                 rotation_range=10,                                       # Data Agumentations
                                 zoom_range=0.15,
                                 height_shift_range=0.5,
                                 horizontal_flip=True,
                                 validation_split=0.2)

train_generator=train_datagen.flow_from_directory("./scrapped images",
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 shuffle=True,
                                                 subset='training')

validation_generator=train_datagen.flow_from_directory("./scrapped images",
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 shuffle=True,
                                                 subset='validation')

Found 736 images belonging to 2 classes.
Found 184 images belonging to 2 classes.


### Defining Model Architecture

In [ ]:
# Loading the MobileNet model
base_model = MobileNetV2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
) 

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(224, 224, 3))


# The base model contains batchnorm layers. We want to keep them in inference 
# mode when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1,activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


### Model Training

In [ ]:
#last layers training
model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()]
)

epochs = 25

reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, mode='auto')
early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=6, mode='auto')
filepath = "./Retrained MobileNet Models/training-model-{epoch:02d}.h5"
model_save = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', save_freq=5)

model.fit(x=train_generator,
          steps_per_epoch = train_generator.n//train_generator.batch_size,
          validation_data = validation_generator, 
          validation_steps = validation_generator.n//validation_generator.batch_size,
          epochs = epochs,
          callbacks=[reduce, early, model_save])


Epoch 1/25
23/23 [==============================] - 167s 7s/step - loss: 0.8117 - binary_accuracy: 0.5545 - val_loss: 0.7418 - val_binary_accuracy: 0.5125
Epoch 2/25
23/23 [==============================] - 14s 608ms/step - loss: 0.7212 - binary_accuracy: 0.5715 - val_loss: 0.7093 - val_binary_accuracy: 0.5813
Epoch 3/25
23/23 [==============================] - 14s 600ms/step - loss: 0.7484 - binary_accuracy: 0.5393 - val_loss: 0.7111 - val_binary_accuracy: 0.5688
Epoch 4/25
23/23 [==============================] - 14s 615ms/step - loss: 0.6817 - binary_accuracy: 0.5677 - val_loss: 0.6789 - val_binary_accuracy: 0.5688
Epoch 5/25
23/23 [==============================] - 14s 608ms/step - loss: 0.6534 - binary_accuracy: 0.6290 - val_loss: 0.6722 - val_binary_accuracy: 0.5688
Epoch 6/25
23/23 [==============================] - 14s 597ms/step - loss: 0.6587 - binary_accuracy: 0.6312 - val_loss: 0.6457 - val_binary_accuracy: 0.6250
Epoch 7/25
23/23 [==============================] - 14s 608m

### Model Fine Tuning
By training all layers a bit

In [ ]:
# fine tuning
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-6),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()]
)

reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, mode='auto')
early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=4, mode='auto')
filepath = "./Retrained MobileNet Models/Final-model-{epoch:02d}.h5"
model_save = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)

epochs = 10
model.fit_generator(generator=train_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    validation_data = validation_generator, 
                    validation_steps = validation_generator.n//validation_generator.batch_size,
                    epochs = epochs,
                    callbacks=[reduce, early, model_save])


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
23/23 [==============================] - 33s 1s/step - loss: 0.4661 - binary_accuracy: 0.7805 - val_loss: 0.4882 - val_binary_accuracy: 0.7750
Epoch 2/10
23/23 [==============================] - 28s 1s/step - loss: 0.4198 - binary_accuracy: 0.8235 - val_loss: 0.4811 - val_binary_accuracy: 0.7875
Epoch 3/10
23/23 [==============================] - 28s 1s/step - loss: 0.4734 - binary_accuracy: 0.7855 - val_loss: 0.4546 - val_binary_accuracy: 0.7812
Epoch 4/10
23/23 [==============================] - 27s 1s/step - loss: 0.4175 - binary_accuracy: 0.8109 - val_loss: 0.4583 - val_binary_accuracy: 0.7688
Epoch 5/10
23/23 [==============================] - 28s 1s/step - loss: 0.4065 - binary_accuracy: 0.8376 - val_loss: 0.4444 - val_binary_accuracy: 0.8000
Epoch 6/10
23/23 [==============================] - 29s 1s/step - loss: 0.4353 - binary_accuracy: 0.7957 - val_loss: 0.4326 - val_binary_accuracy: 0.8062
Epoch 7/10
23/23 [==============================] - 27s 1s/step - loss: 0.40

Due to use of Model checkpoint callback, we saved the models after every epoch. Considering the model after 10th epoch with training accuracy of 81.9 and validation accuracy of 87.5

# Prediction

### Load both the models for predecting

In [5]:
nonOpti_model = load_model('./Retrained MobileNet Models/Final_modelN_89_87.h5')
Opti_model    = load_model('./Retrained MobileNet Models/Final-model-10.h5')

**Function required for prediction**

In [6]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

**From the code below, directly model.predict() will be called on the "bounding box of a car".**   
The results could be interpreted as
* if "<0.5" then Sedan
* if ">0.5" then SUV




In [7]:

# img_path = '/content/gdrive/My Drive/Colab_Notebooks/Case_Studies/MobileNet/cars_test_mini/s1.jpg'
# new_image = load_image(img_path)
# pred = model.predict(new_image)
# print(pred)

print("Non-opti\n")
for i in range(1,8):
  img_path = './Archive/Test images/s'+str(i)+'.jpg'
  new_image = load_image(img_path)
  pred = nonOpti_model.predict(new_image)
  #pred1 = decode_predictions(pred)  
  print("s"+str(i)+": "+str(pred))

for i in range(1,5):
  img_path = './Archive/Test images/se'+str(i)+'.jpg'
  new_image = load_image(img_path)
  pred = nonOpti_model.predict(new_image)
  #pred1 = decode_predictions(pred)
  print("se"+str(i)+": "+str(pred))  

print("\n")
print("Opti\n")
for i in range(1,8):
  img_path = './Archive/Test images/s'+str(i)+'.jpg'
  new_image = load_image(img_path)
  pred = Opti_model.predict(new_image)
  #pred1 = decode_predictions(pred)  
  print("s"+str(i)+": "+str(pred))

for i in range(1,5):
  img_path = './Archive/Test images/se'+str(i)+'.jpg'
  new_image = load_image(img_path)
  pred = Opti_model.predict(new_image)
  #pred1 = decode_predictions(pred)
  print("se"+str(i)+": "+str(pred))    

Non-opti

s1: [[0.7645307]]
s2: [[0.9616349]]
s3: [[0.5487058]]
s4: [[0.9736796]]
s5: [[0.887488]]
s6: [[0.93356997]]
s7: [[0.9238109]]
se1: [[0.11395901]]
se2: [[0.04612371]]
se3: [[0.25387716]]
se4: [[0.11364701]]


Opti

s1: [[0.71382827]]
s2: [[0.94301295]]
s3: [[0.7187693]]
s4: [[0.9572166]]
s5: [[0.6194374]]
s6: [[0.9540255]]
s7: [[0.92153907]]
se1: [[0.14523795]]
se2: [[0.09738412]]
se3: [[0.28262565]]
se4: [[0.1089395]]
